In [1]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
# _set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "🚀Custom_OpenAI_Agent"



## Imports

In [2]:
from openai import AsyncOpenAI
from langsmith.run_helpers import traceable
import os
import asyncio
import json
import requests
import aiohttp

## OpenAI Setup

In [ ]:
client = AsyncOpenAI()

In [ ]:
messages = [
    {
      "role": "system",
      "content": "You are a helpful assistant. Only use the functions you have been provided with.",
    },
    ## Add other messages as needed
  ]



In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "latitude": {
            "type": "string",
          },
          "longitude": {
            "type": "string",
          },
        },
        "required": ["longitude", "latitude"],
      },
    }
  },
  {
    "type": "function",
    "function": {
      "name": "getLocation",
      "description": "Get the user's location based on their IP address",
      "parameters": {
        "type": "object",
        "properties": {},
      },
    }
  },
  {
    "type": "function",
    "function": {
      "name": "sendEmail",
      "description": "A tool to send an email to the user",
      "parameters": {
          "type": "object",
          "properties": {
          "content": {
            "type": "string",
          },
          "subject": {
            "type": "string",
          },
          "recipients": {
            "type": "array",
            "items": {
              "type": "string"
            }
          }
        },
        "required": ["content", "subject", "recipients"]
      },
    }
  },
  {
    "type": "function",
    "function": {
      "name": "htmlContentGenerator",
      "description": "A tool to convert text to HTML",
      "parameters": {
        "type": "object",
        "properties": {
          "content": {
            "type": "string",
          }
        },
        "required": ["content"]
      },
    }
  }
]

In [ ]:
@traceable(run_type="llm",name="OpenAICall")
async def openai_call(model: str = "gpt-4o", messages: list = messages, tools: list = tools):
    try:
        response = await client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
    )
        return response
    except Exception as e:
        print(f"Error during OpenAI call")
        print(f"Exception: {e}")
        return e



## Tools

In [ ]:
from typing import Annotated
import resend
from dotenv import load_dotenv

load_dotenv()

async def getLocation():
    async with aiohttp.ClientSession() as session:
        async with session.get("https://ipapi.co/json/") as response:
            location_data = await response.json()
    return location_data

async def getCurrentWeather(latitude: str, longitude: str):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=apparent_temperature"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            weather_data = await response.json()
    return weather_data


async def sendEmail(
            content: Annotated[str, "The content of the email in HTML format"],
            subject: Annotated[str, "The subject of the email"] = "Hello world",
            recipients: Annotated[list[str], "The list of recipients"] = ["ibrahim.aka.ajax@gmail.com"]
            ):
        """
        This tool is used to send an email to the user. 
        The email should be in HTML format.
        So, if neede
        """
        try:
            resend.api_key = os.getenv("RESEND_API_KEY")
            params: resend.Emails.SendParams = {
                "from": "Acme <onboarding@resend.dev>",
                "to": recipients,
                "subject": subject,
                "html": content,
            }
            return resend.Emails.send(params)
        except Exception as e:
            return repr(e)    


## Tools continued

In [ ]:
from pydantic import BaseModel, Field
from typing import Type
from openai import OpenAI

client = OpenAI()


class HTMLContent(BaseModel):
    language: str = Field(
        description="The language of the content")
    content: str = Field(
        description="The content to be converted to HTML")

#TODO: Use instructor to extract the html content from the response
def htmlContentGenerator(raw_content: str):
    """A tool to convert text to HTML"""
    try:
        response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": f"Convert this text to HTML: {raw_content}"
            }
        ],
        tools=[
        {
            "type": "function",
            "function": {
                "name": "Requirements",
                "description": "Instructions on how to output the HTML content",
                "parameters": HTMLContent.model_json_schema(),
            },
        }
    ],
        tool_choice={
            "type": "function",
            "function": {"name": "Requirements"},
        },
    )
        # response_content = response.choices[0].message.content
        return response
    except Exception as e:
        print(f"Error during OpenAI call")
        print(f"Exception: {e}")
        return e


In [ ]:
availableTools = {
    "getCurrentWeather": getCurrentWeather,
    "getLocation": getLocation
}



## Agent setup

In [ ]:
@traceable(run_type="chain",name="OpenAI-Agent")
async def agent(userInput: str):
    print("Starting agent function...")
    messages.append({
        "role": "user",
        "content": userInput,
    })
    for i in range(5):
        print(f"Iteration {i+1}")
        response = await openai_call()
        print(response)
        finish_reason = response.choices[0].finish_reason
        print(f"Finish reason: {finish_reason}")
        tool_calls_in_message = response.choices[0].message.tool_calls
        print(tool_calls_in_message)

        if finish_reason == "tool_calls" and tool_calls_in_message:
            function_name = tool_calls_in_message[0].function.name
            print(f"Function name: {function_name}")
            function_args = json.loads(tool_calls_in_message[0].function.arguments)
            print(f"Function arguments: {function_args}")
            function_to_call = availableTools[function_name]
            try:
                # Print messages before function call
                print(f"Messages before function call: {messages}")
                function_response = await function_to_call(**function_args)
                print(f"Function {function_name} returned: {function_response}")
                messages.append({
                    "role": "function",
                    "name": function_name,
                    "content": f"The result of the last function was this: {json.dumps(function_response)}"
                })
                # Print messages after function call
                print(f"Messages after function call: {messages}")
            except Exception as e:
                print(f"Error during function call {function_name}: {str(e)}")
                return f"An error occurred: {str(e)}"
        elif finish_reason == "stop":
            messages.append(response.choices[0].message)
            return response.choices[0].message.content
    return "The maximum number of iterations has been met without a suitable answer. Please try again with a more specific input."



In [ ]:
async def run_agent():
    response = await agent("Please suggest some activities based on my location and the current weather.")
    print(response)


In [ ]:
await run_agent()